In [4]:
import pandas as pd
import numpy as np
import operator
from sklearn import preprocessing

def readTrainFile(filename):
    return pd.read_csv(filename, sep=' ')

def createCrossValSet(original):
    train = original.iloc[:,0:100].copy()
    test = original.iloc[:,100:].copy()
    actual = original.iloc[:,100:].copy()
    for idx in range(len(test)):
        if idx % 2 == 1:
            test.iloc[idx,:] = 'x'
    return train, test, actual
    
def readTestFile(filename):
    return pd.read_csv(filename, sep=' ')

def imputeMode(train, test):
    counts = {0:0, 1:0, 2:0}
    for individual in range(len(train)):
        if train[individual] == 0:
            counts[0] = counts[0] + 1
        elif train[individual] == 1:
            counts[1] = counts[1] + 1
        elif train[individual] == 2:
            counts[2] = counts[2] + 1
    mode = max(counts.items(), key=operator.itemgetter(1))[0]
    return mode

def calculateAccuracy(actual, predicted):
    if actual.shape != predicted.shape:
        print("Mismatch matrix dimensions")
        pass
    sampleSize = actual.shape[0]*actual.shape[1]
    correct = 0
    for SNP in range(len(actual)):
        for individual in range (len(actual.iloc[SNP])):
            if actual.iloc[SNP, individual] == predicted.iloc[SNP, individual]:
                correct = correct + 1
    return correct/sampleSize

def baselineImputation(train, test):
    # Simple Mean Imputation: take the mode of each SNP across all IND_#
    # This assumes SNPs are independent
    for SNP in range(len(train)):
        test.iloc[SNP,:] = test.iloc[SNP,:].replace('x', imputeMode(train.iloc[SNP,:], test))
        

if __name__ == "__main__":
#     trainSNPs = readTrainFile('imputation_training.txt')
#     testSNPs = readTestFile('imputation_test.txt')
    original = pd.read_csv('imputation_training.txt', sep=' ')
    sets = [100, 1000, 10000]
    for SNPcount in sets:
        train, test, actual = createCrossValSet(original.iloc[0:SNPcount,:])
        baselineImputation(train, test)
        print(str(SNPcount)+' SNPs Baseline Accuracy: '+str(calculateAccuracy(actual, test)))

100 SNPs Baseline Accuracy: 0.8822857142857143
1000 SNPs Baseline Accuracy: 0.8262285714285714
10000 SNPs Baseline Accuracy: 0.8078542857142857
